# Design Pattern 6: Multilabel

> Refere-se a problemas em que um mesmo exemplo pertence a mais de uma classe.

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from typing import List

from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import hamming_loss

import warnings
warnings.filterwarnings('ignore')

### Base de exemplo

In [2]:
df = pd.read_csv('data/sf_robots.csv', usecols=['Original_Book_Title', 'Book_Description', 'Genres']).dropna()
print(df.shape)
df.head(1)

(1239, 3)


,Original_Book_Title,Book_Description,Genres
0,"I, Robot","Isaac Asimov's I, Robot launches readers on an...","{'Science Fiction': 6502, 'Fiction': 2523, 'Cl..."


#### Pré-processamento

In [3]:
def getGenres(genres: dict) -> str:
    keys = list(genres.keys())
    return ';'.join(keys)

df['Genres'] = df['Genres'].apply(eval)
df['Genres'] = df['Genres'].apply(getGenres)

In [4]:
count = df['Genres'].value_counts()
valuesWithMoreThan5Examples = count[count >= 2].index
sample = df[df['Genres'].isin(valuesWithMoreThan5Examples)]
sample = sample.query('Genres!="None"')

### Design Pattern 6: Multilabel

- Usamos a abordagem one vs. rest que é uma técnica que consaiste em treinar um modelo binário para cada rótulo e é bom em lidar com classes raras.

#### Converter os rótulos

In [5]:
mlb = MultiLabelBinarizer()
sample['Genres'] = sample['Genres'].apply(lambda x: [genre.strip() for genre in x.split(';')])
y = mlb.fit_transform(sample['Genres'])

#### Separar dados de treinamento e teste

In [6]:
X_train, X_test, y_train, y_test = train_test_split(sample['Book_Description'],
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

#### Multilabel one vs. rest

In [7]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
    ('classifier', OneVsRestClassifier(LogisticRegression()))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Quanto menor o valor do Hamming Loss, melhor é o desempenho do modelo.
hamming_loss_value = hamming_loss(y_test, y_pred)

print(f'Taxa média de erro na predição dos rótulos: {hamming_loss_value:.2f}')
print(f'Em média, {round(hamming_loss_value*100)}% dos rótulos atribuídos pelo modelo estão incorretos.')

Taxa média de erro na predição dos rótulos: 0.09
Em média, 9% dos rótulos atribuídos pelo modelo estão incorretos.


#### Teste

In [8]:
book = df['Original_Book_Title'][606]
genre = df['Genres'][606].strip()
print(f'Para o primeiro exemplo de teste temos o livro *{book}* que, originalmente, pertence ao(s) gênero(s) *{genre}*')

Para o primeiro exemplo de teste temos o livro *Mobile Robots: Inspiration to Implementation* que, originalmente, pertence ao(s) gênero(s) *Science Fiction (Robots)*


In [9]:
pred = pipeline.predict(X_test[:1].values)
label = mlb.inverse_transform(pred)[0][0]
print(f'Para esse livro, nosso modelo previu que ele pertence ao(s) gênero(s) {label}')

Para esse livro, nosso modelo previu que ele pertence ao(s) gênero(s) Science Fiction (Robots)
